In [16]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.nn.functional as F

# Set random seed for reproducibility
np.random.seed(67)

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [17]:
# Get device for training
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [19]:
# Load dataset
housing = fetch_california_housing()
X, Y = housing.data, housing.target
num_classes = len(np.unique(Y))
print(f"Dataset shape: {X.shape}")
print(f"Number of classes: {num_classes}")
print(f"Features per sample: {X.shape[1]}")

Dataset shape: (20640, 8)
Number of classes: 3842
Features per sample: 8


In [10]:
# Data processing parameters
data_portions = (0.6,0.2,0.2)
training_portion, validation_portion, test_portion = data_portions
# Normalize
validation_portion = validation_portion / (validation_portion+training_portion)

In [14]:
# Normalize inputs
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_normalized = (X - X_mean) / X_std

# Split into training and test sets
num_samples = X_normalized.shape[0]
X_temp, X_test, Y_temp, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=20
)

X_train, X_val, Y_train, Y_val = train_test_split(
    X_temp, Y_temp, test_size=0.25, random_state=20
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Fit on training data only!
X_val = scaler.transform(X_val)          # Use training statistics
X_test = scaler.transform(X_test)        # Use training statistics

In [28]:
class Net(nn.Module):

    def __init__(self, input_size, output_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size,64),
            nn.ReLU(),
            nn.Linear(64,128),
            nn.ReLU(),
            nn.Linear(128,output_size),
            nn.Softmax()
        )
    
    def forward(self,x):
        return self.net(x)
    
    def predict(self,x):
        Y_pred = self.forward(x)
        return Y_pred

In [ ]:
def create_mini_batches(X, y, batch_size):
    """Create mini-batches for training."""
    m = X.shape[0]
    mini_batches = []
    
    # Shuffle data
    permutation = np.random.permutation(m)
    X_shuffled = X[permutation]
    y_shuffled = y[permutation]
    
    # Create mini-batches
    num_complete_batches = m // batch_size
    
    for i in range(num_complete_batches):
        X_batch = X_shuffled[i * batch_size:(i + 1) * batch_size]
        y_batch = y_shuffled[i * batch_size:(i + 1) * batch_size]
        mini_batches.append((X_batch, y_batch))
    
    # Handle remaining samples
    if m % batch_size != 0:
        X_batch = X_shuffled[num_complete_batches * batch_size:]
        y_batch = y_shuffled[num_complete_batches * batch_size:]
        mini_batches.append((X_batch, y_batch))
    
    return mini_batches

def train_model(model, X_train, Y_train, X_val, Y_val, num_epochs, batch_size, learning_rate):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': []
    }
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        mini_batches = create_mini_batches(X_train, Y_train, batch_size)
        for i, (X_batch, Y_batch) in enumerate(mini_batches):
             # Forward pass
            outputs = model(torch.tensor(X_batch, dtype=torch.float32).to(device))
            Y_batch = torch.tensor(Y_batch, dtype=torch.float32).to(device)
            loss = criterion(outputs, Y_batch)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            
        avg_epoch_loss = epoch_loss / len(mini_batches)
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(torch.tensor(X_val, dtype=torch.float32).to(device))
            Y_val_tensor = torch.tensor(Y_val, dtype=torch.float32).to(device).view(-1, 1)
            val_loss = criterion(val_outputs, Y_val_tensor).item()
            
            # Calculate training accuracy
            train_preds = model(torch.tensor(X_train, dtype=torch.float32).to(device))
            train_preds_labels = torch.argmax(train_preds, dim=1).cpu().numpy()
            train_acc = np.mean(train_preds_labels == Y_train)
            
            # Calculate validation accuracy
            val_preds_labels = torch.argmax(val_outputs, dim=1).cpu().numpy()
            val_acc = np.mean(val_preds_labels == Y_val)

        # record history
        history['train_loss'].append(epoch_loss/len(mini_batches))
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)

        # print progress
        if (epoch+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], '
                   f'Train Loss: {avg_epoch_loss:.4f}, '
                   f'Val Loss: {val_loss:.4f}, '
                   f'Train Acc: {train_acc:.4f}, '
                   f'Val Acc: {val_acc:.4f}')

    return history

In [32]:
# Configure model
input_size = X_train.shape[1]
output_size = 1  # Regression output

In [ ]:
model = Net(input_size, output_size).to(device)

history = train_model(model, X_train=X_train, Y_train=Y_train, X_val=X_val, Y_val=Y_val, num_epochs=500, 
                      batch_size=32, learning_rate=0.001)

# Evaluate on test set
model.eval()
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32).to(device).view(-1, 1)
    Y_pred = model.predict(X_test_tensor)
    test_loss = nn.MSELoss()(Y_pred, Y_test_tensor)
    print(f'Test MSE Loss: {test_loss.item():.4f}')

Epoch [10/500], Train Loss: 2.4763, Val Loss: 2.3948, Train Acc: 0.0000, Val Acc: 0.0000
Epoch [20/500], Train Loss: 2.4763, Val Loss: 2.3948, Train Acc: 0.0000, Val Acc: 0.0000
Epoch [30/500], Train Loss: 2.4763, Val Loss: 2.3948, Train Acc: 0.0000, Val Acc: 0.0000
Epoch [40/500], Train Loss: 2.4763, Val Loss: 2.3948, Train Acc: 0.0000, Val Acc: 0.0000
Epoch [50/500], Train Loss: 2.4763, Val Loss: 2.3948, Train Acc: 0.0000, Val Acc: 0.0000
Epoch [60/500], Train Loss: 2.4763, Val Loss: 2.3948, Train Acc: 0.0000, Val Acc: 0.0000
Epoch [70/500], Train Loss: 2.4763, Val Loss: 2.3948, Train Acc: 0.0000, Val Acc: 0.0000


KeyboardInterrupt: 